## Library

In [ ]:
import os
import csv
import numpy as np
import pandas as pd
import cv2
import pydicom
from tqdm import tqdm
from skimage.transform import resize
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import Optimizer
from torch.utils.data import Dataset, DataLoader
from torch.amp import GradScaler, autocast
from torchvision import transforms
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torchvision.models as models

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

## Init GPU

In [ ]:
# Initialize GPU Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")

print(device)

In [ ]:
%load_ext autoreload
%autoreload 2

## Config Info

In [ ]:
# Constants
TEST_SIZE = 0.02
HEIGHT = 512
WIDTH = 512
CHANNELS = 2
TRAIN_BATCH_SIZE = 2
VALID_BATCH_SIZE = 2
TEST_BATCH_SIZE = 2
MAX_SLICES = 32
NUM_EPOCHS = 2
ACCUMULATION_STEPS = 64
SHAPE = (HEIGHT, WIDTH, CHANNELS)

In [ ]:
# Folders
DATA_DIR = '/kaggle/input/rsna-mil-training/'
DICOM_DIR = DATA_DIR + 'rsna-mil-training'
CSV_PATH = DATA_DIR + 'training_1000_scan_subset.csv'
patient_scan_labels = pd.read_csv(CSV_PATH)

In [ ]:
def preprocess_slice(slice, target_size=(HEIGHT, WIDTH)):
    slice = resize(slice, target_size, anti_aliasing=True)
    brain_channel = apply_windowing(slice, window=(40, 80))
    subdural_channel = apply_windowing(slice, window=(80, 200))
    bone_channel = apply_windowing(slice, window=(600, 2800))
    
    multichannel_slice = np.stack([brain_channel, subdural_channel, bone_channel], axis=-1)
    return multichannel_slice.astype(np.float16)  # Use float16 for reduced memory usage

def apply_windowing(slice, window):
    window_width, window_level = window
    lower_bound = window_level - window_width // 2
    upper_bound = window_level + window_width // 2
    
    windowed_slice = np.clip(slice, lower_bound, upper_bound)
    windowed_slice = (windowed_slice - lower_bound) / (upper_bound - lower_bound)
    return windowed_slice

In [ ]:
def read_dicom_folder(folder_path):
    slices = []
    for filename in sorted(os.listdir(folder_path))[:MAX_SLICES]:  # Limit to MAX_SLICES
        if filename.endswith(".dcm"):
            file_path = os.path.join(folder_path, filename)
            ds = pydicom.dcmread(file_path)
            slices.append(ds.pixel_array)
    
    # Pad with black images if necessary
    while len(slices) < MAX_SLICES:
        slices.append(np.zeros_like(slices[0]))
    
    return slices[:MAX_SLICES]  # Ensure we return exactly MAX_SLICES

In [ ]:
def process_patient_data(dicom_dir, row):
    """
    Process data for a single patient based on the row from the DataFrame.
    
    Args:
        dicom_dir (str): The directory containing DICOM folders.
        row (pd.Series): A row from the patient_scan_labels DataFrame.

    Returns:
        Tuple: Preprocessed slices and label.
    """
    patient_id = row['patient_id'].replace('ID_', '')  # Remove 'ID_' prefix
    study_instance_uid = row['study_instance_uid'].replace('ID_', '')  # Remove 'ID_' prefix
    
    # Construct folder path based on patient_id and study_instance_uid
    folder_name = f"{patient_id}_{study_instance_uid}"
    folder_path = os.path.join(dicom_dir, folder_name)
    
    # Read and preprocess DICOM slices
    if os.path.exists(folder_path):
        slices = read_dicom_folder(folder_path)
        preprocessed_slices = [preprocess_slice(slice) for slice in slices]
        
        # Determine label based on any of the hemorrhage indicators
        label = 1 if row[['any', 'epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural']].any() else 0
        
        return preprocessed_slices, label
    else:
        print(f"Folder not found: {folder_path}")
        return None, None  # Handle the case where the folder is not found

In [ ]:
class TrainDatasetGenerator(Dataset):
    def __init__(self, dicom_dir, patient_scan_labels):
        self.dicom_dir = dicom_dir
        self.patient_scan_labels = patient_scan_labels
        self.transform = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
            transforms.ColorJitter(brightness=0.2, contrast=0.2)
        ])

    def __len__(self):
        return len(self.patient_scan_labels)

    def __getitem__(self, idx):
        row = self.patient_scan_labels.iloc[idx]
        preprocessed_slices, label = process_patient_data(self.dicom_dir, row)
        
        if preprocessed_slices is not None:
            preprocessed_slices = np.array(preprocessed_slices)
            preprocessed_slices = torch.tensor(preprocessed_slices, dtype=torch.float32)  # Use float16 for mixed precision
            
            # Apply augmentations
            augmented_slices = []
            for slice in preprocessed_slices:
                augmented_slice = self.transform(slice.permute(2, 0, 1)).permute(1, 2, 0)
                augmented_slices.append(augmented_slice)
            
            return torch.stack(augmented_slices), torch.tensor(label, dtype=torch.long)
        else:
            return None, None

class TestDatasetGenerator(Dataset):
    def __init__(self, dicom_dir, patient_scan_labels):
        self.dicom_dir = dicom_dir
        self.patient_scan_labels = patient_scan_labels
        # No augmentations for test data, but we'll use the same preprocessing
        self.transform = transforms.Compose([
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.patient_scan_labels)

    def __getitem__(self, idx):
        row = self.patient_scan_labels.iloc[idx]
        preprocessed_slices, label = process_patient_data(self.dicom_dir, row)

        if preprocessed_slices is not None:
            preprocessed_slices = np.array(preprocessed_slices)
            preprocessed_slices = torch.tensor(preprocessed_slices, dtype=torch.float16)  # Use float16 for mixed precision
            
            # Apply normalization
            normalized_slices = []
            for slice in preprocessed_slices:
                normalized_slice = self.transform(slice.permute(2, 0, 1)).permute(1, 2, 0)
                normalized_slices.append(normalized_slice)
            
            file_names = [f"{row['patient_id']}_{row['study_instance_uid']}"]
            return torch.stack(normalized_slices), torch.tensor(label, dtype=torch.long), file_names
        else:
            return None, None, None

In [ ]:
def get_train_loader(dicom_dir, patient_scan_labels, batch_size=TRAIN_BATCH_SIZE, shuffle=True):
    train_dataset = TrainDatasetGenerator(dicom_dir, patient_scan_labels)
    return DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=4, pin_memory=True)

def get_test_loader(dicom_dir, patient_scan_labels, batch_size=TEST_BATCH_SIZE):
    test_dataset = TestDatasetGenerator(dicom_dir, patient_scan_labels)
    return DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

## CNN Feature Extractor

In [ ]:
class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        efficientnet = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
        self.features = nn.Sequential(*list(efficientnet.children())[:-1])
        
    def forward(self, x):
        x = self.features(x)
        return x.view(x.size(0), -1)

In [ ]:
class AttentionLayer(nn.Module):
    def __init__(self, input_dim, attention_dim):
        super(AttentionLayer, self).__init__()
        self.attention = nn.Sequential(
            nn.Linear(input_dim, attention_dim),
            nn.Tanh(),
            nn.Linear(attention_dim, 1)
        )
    
    def forward(self, features):
        weights = self.attention(features)
        weights = torch.softmax(weights, dim=1)
        weighted_features = torch.sum(weights * features, dim=1)
        return weighted_features, weights

In [ ]:
class MILModel(nn.Module):
    def __init__(self, num_classes):
        super(MILModel, self).__init__()
        self.feature_extractor = FeatureExtractor()
        self.attention = AttentionLayer(1280, 256)  # EfficientNet-B0 outputs 1280 features
        self.classifier = nn.Sequential(
            nn.Linear(1280, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        batch_size, num_images, channels, height, width = x.size()
        x = x.view(-1, channels, height, width)
        features = self.feature_extractor(x)
        features = features.view(batch_size, num_images, -1)
        weighted_features, attention_weights = self.attention(features)
        output = self.classifier(weighted_features)
        return output, attention_weights, features

In [ ]:
def train_model(model, train_loader, criterion, optimizer, scaler, device, num_epochs, accumulation_steps, scheduler=None):
    for epoch in range(num_epochs):
        # Training Phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        train_loader_tqdm = tqdm(train_loader, leave=False)
        optimizer.zero_grad()
        
        for i, (inputs, labels) in enumerate(train_loader_tqdm):
            inputs, labels = inputs.to(device), labels.to(device)
            inputs = inputs.permute(0, 1, 4, 2, 3)  # Rearrange dimensions
            
            with autocast(device.type):
                outputs, _, _ = model(inputs)
                outputs = outputs.squeeze(1)
                loss = criterion(outputs, labels.float()) / accumulation_steps
            
            scaler.scale(loss).backward()
            
            if (i + 1) % accumulation_steps == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
            
            running_loss += loss.item()
            predicted = (outputs >= 0.5).float()
            correct += (predicted == labels.float()).sum().item()
            total += labels.size(0)
            
            train_loader_tqdm.set_description(f"Epoch {epoch+1}/{num_epochs} - Loss: {loss.item():.4f}")
        
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct / total
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}')
        
        if scheduler:
            scheduler.step(val_loss)
    
    return model

In [ ]:
def evaluate_model(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    all_file_names = []  # To store DICOM file names
    
    with torch.no_grad():
        for inputs, labels, file_names in tqdm(test_loader):
            if inputs is None or labels is None or file_names is None:
                continue

            inputs, labels = inputs.to(device), labels.to(device)
            inputs = inputs.permute(0, 1, 4, 2, 3)  # Permute to match model input shape
            
            with autocast(device.type):
                outputs, _, _ = model(inputs)
                outputs = outputs.squeeze(1)
                loss = criterion(outputs, labels.float())
            
            running_loss += loss.item()
            
            # Adjust threshold for binary classification
            predicted = (outputs >= 0.3).float()  # Changed threshold from 0.5 to 0.3
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_file_names.extend(file_names)  # Collect file names
    
    avg_loss = running_loss / len(test_loader)
    accuracy = accuracy_score(all_labels, all_preds)
    
    # Handle undefined metrics by using zero_division=1 to avoid warnings
    precision = precision_score(all_labels, all_preds, zero_division=1)
    recall = recall_score(all_labels, all_preds, zero_division=1)
    f1 = f1_score(all_labels, all_preds, zero_division=1)
    
    print(f"Test Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")
    
    return all_preds, all_labels, all_file_names, avg_loss, accuracy, precision, recall, f1

In [ ]:
def save_test_results(file_names, predictions, labels, output_file="test_results.csv"):
    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["DICOM File", "True Label", "Predicted Label"])  # Add DICOM file as the first column
        for file_name, true_label, pred in zip(file_names, labels, predictions):
            writer.writerow([file_name, true_label, pred])
    
    print(f"Results saved to {output_file}")

In [ ]:
if __name__ == "__main__":
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    train_loader = get_train_loader(DICOM_DIR, patient_scan_labels, batch_size=TRAIN_BATCH_SIZE)
    test_loader = get_test_loader(DICOM_DIR, patient_scan_labels, batch_size=TEST_BATCH_SIZE)

    model = MILModel(num_classes=1).to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=1e-5)  # Reduced learning rate
    scaler = GradScaler()

    trained_model = train_model(model, train_loader, criterion, optimizer, scaler, device, NUM_EPOCHS, ACCUMULATION_STEPS)

    torch.save(trained_model.state_dict(), 'optimized_mil_model_p100.pth')  
    
    print('Training completed. Evaluating...')
    
    model.load_state_dict(torch.load('optimized_mil_model_p100.pth', weights_only=True))
    predictions, labels, file_names, test_loss, test_acc, test_precision, test_recall, test_f1 = evaluate_model(model, test_loader, criterion, device)
    
    save_test_results(file_names, predictions, labels)